# Deep Research

In [4]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv

import asyncio
import os

from typing import Dict
from IPython.display import display, Markdown

In [5]:
load_dotenv(override=True)

True

In [6]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [7]:
INSTRUCTIONS = "You are a research assistant. Given the search term, you search the web for that term and \
    produce concise summary of the results. The summary must have 2-3 paragraphs and less than 300 words. \
    Capture the main points. Write succinctly, not need to have complete sentences or good grammar. This will \
    be consumed by someone synthesizing a report, so it's vital you capture the essence and ignore any fluff. \
    Do not include any additional commentary other than the summary itself."
    
search_agent = Agent(
    name='Search Agent',
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size='low')],
    model='gpt-4o-mini',
    model_settings=ModelSettings(tool_choice='required')
)

In [5]:
message = "Latest AT Agent frameworks in 2025"

with trace('WebSearch'):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

In 2025, several AI agent frameworks have emerged, each offering unique features for developing intelligent systems:

- **LangChain**: A modular framework for building applications with large language models (LLMs), addressing challenges like context retention and multi-step task execution. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **LangGraph**: An extension of LangChain, focusing on stateful, multi-actor systems, ideal for complex workflows and adaptive AI applications. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **CrewAI**: Designed for role-based AI agents, facilitating collaborative problem-solving environments requiring diverse expertise. ([linkedin.com](https://www.linkedin.com/pulse/top-5-frameworks-building-ai-agents-2025-sahil-malhotra-wmisc?utm_source=openai))

- **AutoGen**: Developed by Microsoft, it enables advanced multi-agent conversations, tool usage, and memory, integrated with Microsoft's Semantic Kernel and Azure services for secure enterprise deployment. ([medium.com](https://medium.com/%40rajadityasatellite/2025-is-the-year-of-ai-agent-frameworks-cb24e3f9ffc7?utm_source=openai))

- **SuperAGI**: An open-source agent framework designed for autonomy, including scheduling, execution monitoring, performance dashboards, and persistent agent memory. ([medium.com](https://medium.com/%40rajadityasatellite/2025-is-the-year-of-ai-agent-frameworks-cb24e3f9ffc7?utm_source=openai))

- **AgentCore**: Introduced by AWS, this platform supports enterprise-scale AI agent development and deployment, offering tools for smarter, safer, and more intuitive AI agents. ([techradar.com](https://www.techradar.com/pro/we-want-aws-to-be-the-place-where-everyone-runs-enterprise-ai-agents-the-agentic-era-is-here-for-your-business-so-be-prepared-for-the-new-age?utm_source=openai))

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of LLMs for any AI agent, decoupling agent execution from training for seamless integration. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **AutoAgent**: A fully-automated, zero-code framework for LLM agents, allowing users to create and deploy agents through natural language alone. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **AgentLite**: A lightweight library for building and advancing task-oriented LLM agent systems, simplifying the creation and evaluation of new reasoning strategies and agent architectures. ([arxiv.org](https://arxiv.org/abs/2402.15538?utm_source=openai))

- **Autono**: A robust autonomous agent framework based on the ReAct paradigm, designed for complex tasks through adaptive decision-making and multi-agent collaboration. ([arxiv.org](https://arxiv.org/abs/2504.04650?utm_source=openai))

These frameworks cater to various needs, from enterprise solutions to research applications, reflecting the rapid advancements in AI agent development. 

# Now with usage of Structured Output

In [9]:
HOW_MANY_SEARCHES = 10

INSTRUCTIONS = f"You are a helpful assistant in research. Given the query, come up with a set of web searches \
    to perform to answer the query as best as you can. Output {HOW_MANY_SEARCHES} terms to query for."
    
class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query"
    
    query: str
    "The search term to /use for the web search"
    
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    "A list of web searches to perform to best answer the query"
    
planner_agent = Agent(
    name='PlannerAgent',
    instructions=INSTRUCTIONS,
    model='gpt-4o-mini',
    output_type=WebSearchPlan
)

In [10]:
from loguru import logger

message = "Latest AI Agent frameworks in 2025"

with trace("Search v2"):
    result = await Runner.run(planner_agent, message)

logger.info(result.final_output)

2025-08-13 14:15:55.055 | INFO     | __main__:<module>:8 - searches=[WebSearchItem(reason='To find the most current AI agent frameworks released or announced in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To get insights on the features and capabilities of upcoming AI agent frameworks.', query='AI agent frameworks features 2025'), WebSearchItem(reason='To explore industry reports or analyses on AI frameworks developing in 2025.', query='2025 AI frameworks industry report'), WebSearchItem(reason='To find user reviews or discussions about new AI frameworks.', query='AI agent framework reviews 2025'), WebSearchItem(reason='To identify major tech companies working on AI agent frameworks in 2025.', query='companies developing AI frameworks 2025'), WebSearchItem(reason='To find academic papers or publications related to AI agent frameworks in 2025.', query='AI agent frameworks research papers 2025'), WebSearchItem(reason='To discover comparisons between different 

In [11]:
import requests

@function_tool
def push(
    message: str, 
    device: str = 'iphone', 
    timeout: int = 5) -> bool:
    """Send a push notification with given content, i.ex. a mail template.

    Args:
        message (str): Content of the message being sent
        device (str, optional): Name of the device to send notification to. Defaults to 'iphone'.
        timeout (int, optional): Timeout value. Defaults to 5.

    Returns:
        bool: True if succeeded, False on every other circumstance
    """
    if not all([pushover_user, pushover_token, pushover_url]):
        logger.error("Pushover configuration is missing.")
        return False

    logger.info("Sending push notification...")
    
    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message,
    }
    if device:
        payload["device"] = device

    try:
        response = requests.post(pushover_url, data=payload, timeout=timeout)
        if response.status_code != 200:
            logger.error(f"Pushover failed: {response.status_code} {response.text}")
            return False
        logger.info("Push notification sent successfully.")
        return True
    except requests.RequestException as e:
        logger.exception(f"Error sending push notification: {e}")
        return False

In [12]:
PUSH_INSTRUCTIONS = "You are able to send nicely formatted push notifications based on a detailed report. \
    You will be provided with a detailed report. You should use your tool to send one email, providing \
    the report converted into clean, wel presented HTML with an appropriate subject line."
    
push_agent = Agent(
    name='PushAgent',
    instructions=PUSH_INSTRUCTIONS,
    tools=[push],
    model="gpt-4o-mini"
)
        

In [13]:
RESEARCH_INSTRUCTION = "You are a senior research tasked with writing a cohesive report for a research query. \
    You will be provided with an original query and some initial research done by a research assistant. \
    You should first come up with an outline for the report that describes the structure and overall flow \
    of the report. Then, generate the report and return that as your final output. The final output should be \
    in the Markdown format and it should be lengthy and detailed. Aim for 5-10 pages of content, at least 1000 \
    words."
    
class ReportData(BaseModel):
    short_summary: str
    "A short 3-4 sentence summary of the findings"
    
    markdown_report: str
    "The final report"
    
    followup_suggestions: list[str]
    "Suggested topics to research further"
    
writer_agent = Agent(
    name='WriterAgent',
    instructions=RESEARCH_INSTRUCTION,
    model='gpt-4o-mini',
    output_type=ReportData
)

## Methods to plan, execute and perform the search

In [14]:
async def plan_searches(query: str):
    "Use the planner_agent to plan which searches to run for a query"
    logger.info(f"Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    logger.info(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    "Call search() for each item in the search plan"
    logger.info(f"Searching...")
    num_completed = 0
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    logger.info(f"Finished searching")
    return results

async def search(item: WebSearchItem):
    "Use the search agent to run a web search for each time item in the search plan"
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

## Methods to write and publish the report

In [15]:
async def write_report(query: str, search_results: list[str]):
    "Use the writer agent to write a report based on the search results"
    logger.info(f"Thinking about reports...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    logger.info(f"Finished writing a report")
    return result.final_output

async def send_notification(report: ReportData):
    "Use push agent to send a notification with the report"
    logger.info("Sending a notification...")
    result = await Runner.run(push_agent, report.markdown_report)
    logger.info("Report sent")
    return report

In [16]:
query = "Spółki giełdowe w jakie warto intestować w 2025 na GPW"

with trace("Research Agent trace"):
    logger.info(f"Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    report_data = await send_notification(report)
    logger.info("Workflow done")

2025-08-13 14:16:06.184 | INFO     | __main__:<module>:4 - Starting research...
2025-08-13 14:16:06.185 | INFO     | __main__:plan_searches:3 - Planning searches...
2025-08-13 14:16:13.425 | INFO     | __main__:plan_searches:5 - Will perform 10 searches
2025-08-13 14:16:13.425 | INFO     | __main__:perform_searches:10 - Searching...
2025-08-13 14:16:28.963 | INFO     | __main__:perform_searches:14 - Finished searching
2025-08-13 14:16:28.963 | INFO     | __main__:write_report:3 - Thinking about reports...
2025-08-13 14:16:55.618 | INFO     | __main__:write_report:6 - Finished writing a report
2025-08-13 14:16:55.620 | INFO     | __main__:send_notification:11 - Sending a notification...
2025-08-13 14:17:23.167 | INFO     | __main__:push:22 - Sending push notification...
2025-08-13 14:17:23.997 | INFO     | __main__:push:37 - Push notification sent successfully.
2025-08-13 14:17:25.812 | INFO     | __main__:send_notification:13 - Report sent
2025-08-13 14:17:25.813 | INFO     | __main__:

In [17]:
with open('report_data_v2.md', 'w', encoding='utf-8') as f:
    f.write(report_data.markdown_report)

In [1]:
with open('report_data_v2.md', 'r', encoding='utf-8') as f:
    data = f.read()